In [ ]:
from __future__ import division, print_function

import os
import sys
_path = os.path.abspath("../scripts")
if _path not in sys.path:
    sys.path.insert(0, _path)
from config import galcen_frame, potentials

# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.stats import norm
import scipy.optimize as so
import emcee

# Custom
from gary.util import atleast_2d
import gary.coordinates as gc
import gary.dynamics as gd
from gary.dynamics import mockstream
from gary.dynamics import orbitfit
import gary.integrate as gi
import gary.potential as gp
from gary.units import galactic

## Set the potential to run:

In [ ]:
name = 'triaxial'
potential = potentials[name]
potential

### Should finish at the same place in both potentials:

In [ ]:
end_c = coord.Galactic(l=125*u.degree, b=31*u.degree, distance=15*u.kpc)
end_v = [2.1*u.mas/u.yr, -0.25*u.mas/u.yr, -100*u.km/u.s]

In [ ]:
end_c_gc = end_c.transform_to(galcen_frame['galactocentric_frame'])
end_v_gc = gc.vhel_to_gal(end_c, end_v[:2], end_v[2], **galcen_frame)
w0 = gd.CartesianPhaseSpacePosition(pos=end_c_gc.represent_as(coord.CartesianRepresentation).xyz,
                                    vel=end_v_gc)

In [ ]:
n_steps = 4000
prog_orbit = potential.integrate_orbit(w0, dt=-1., nsteps=n_steps, t1=float(n_steps),
                                       Integrator=gi.DOPRI853Integrator)
prog_orbit = prog_orbit[::-1]
fig = prog_orbit.plot()
fig.axes[0].plot(end_c_gc.represent_as(coord.CartesianRepresentation).xyz[0],
                 end_c_gc.represent_as(coord.CartesianRepresentation).xyz[1],
                 marker='o')

In [ ]:
stream = mockstream.fardal_stream(potential, prog_orbit=prog_orbit, 
                                  prog_mass=5E5*u.Msun, release_every=1, 
                                  Integrator=gi.DOPRI853Integrator)

In [ ]:
prog_c,prog_v = prog_orbit.to_frame(coord.Galactic, **galcen_frame)
stream_c,stream_v = stream.to_frame(coord.Galactic, **galcen_frame)

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(6,6))
ax.plot(stream_c.l.wrap_at(180*u.degree).degree, stream_c.b.degree, ls='none', alpha=0.25)

In [ ]:
pl.plot(prog_c.l.wrap_at(180*u.deg).degree, prog_c.b.degree, marker=None, alpha=0.25, color='b')
pl.scatter(stream_c.l.wrap_at(180*u.deg).degree, stream_c.b.degree, 
           c=stream_c.distance.value, cmap='Greys_r', s=4, vmin=10, vmax=20)
pl.colorbar()

## Select some "stars" to observe

Only select leading tail and most recently stripped stuff

In [ ]:
idx = np.random.permutation(np.arange(n_steps, stream_c.l.size, dtype=int)[::2])[:64]
obs_c = stream_c[idx]
obs_v = [v[idx] for v in stream_v]

# smaller region of sky
idx = (obs_c.l.degree > 105.) & (obs_c.l.degree < 180.) & (obs_c.b.degree > 30)
obs_c = obs_c[idx]
obs_v = [v[idx] for v in obs_v]

In [ ]:
pl.scatter(obs_c.l.wrap_at(180*u.deg).degree, obs_c.b.degree, s=4, c='k')

---

In [ ]:
np.random.seed(42)
R = orbitfit.compute_stream_rotation_matrix(obs_c, align_lon='max')

In [ ]:
# rotate all data to plot
rot_rep = orbitfit.rotate_sph_coordinate(obs_c, R)

pl.figure(figsize=(10,2))
pl.plot(rot_rep.lon.wrap_at(180*u.deg).degree, rot_rep.lat.degree, ls='none', marker='o', ms=5.)
pl.gca().set_aspect('equal')
pl.xlabel(r'$\phi_1$ [deg]')
pl.ylabel(r'$\phi_2$ [deg]')

### Save data

In [ ]:
import pickle

In [ ]:
for frac_distance_err in [1, 5]:
    np.random.seed(42)

    n_data = len(obs_c)
    data = dict()
    err = dict()

    err['distance'] = frac_distance_err/100. * obs_c.distance
    err['mul'] = 0.1*u.mas/u.yr
    err['mub'] = 0.1*u.mas/u.yr
    err['vr'] = 10.*u.km/u.s
    # err['distance'] = 1E-6*obs_c.distance
    # err['mul'] = 1E-1*u.mas/u.yr
    # err['mub'] = 1E-1*u.mas/u.yr
    # err['vr'] = 1E-6*u.km/u.s

    data['phi1'] = rot_rep.lon
    data['phi2'] = rot_rep.lat
    data['distance'] = obs_c.distance + np.random.normal(0., err['distance'].value, size=n_data)*obs_c.distance.unit
    data['mul'] = obs_v[0] + np.random.normal(0., err['mul'].value, size=n_data)*err['mul'].unit
    data['mub'] = obs_v[1] + np.random.normal(0., err['mub'].value, size=n_data)*err['mub'].unit
    data['vr'] = obs_v[2] + np.random.normal(0., err['vr'].value, size=n_data)*err['vr'].unit
    
    with open("../data/{}_{}percent.pickle".format(name, frac_distance_err), 'wb') as f:
        pickle.dump((data,err,R), f)

### Decide on integration time to set

In [ ]:
def _mcmc_sample_to_coord(p, R):
    p = atleast_2d(p, insert_axis=-1) # note: from Gary, not Numpy
    rep = coord.SphericalRepresentation(lon=p[0]*0.*u.radian,
                                        lat=p[0]*u.radian, # this index looks weird but is right
                                        distance=p[1]*u.kpc)
    return coord.Galactic(orbitfit.rotate_sph_coordinate(rep, R.T))

def _mcmc_sample_to_w0(p, R):
    p = atleast_2d(p, insert_axis=-1) # note: from Gary, not Numpy
    c = _mcmc_sample_to_coord(p, R)
    x0 = c.transform_to(galcen_frame['galactocentric_frame']).cartesian.xyz.decompose(galactic).value
    v0 = gc.vhel_to_gal(c, pm=(p[2]*u.rad/u.Myr,p[3]*u.rad/u.Myr), rv=p[4]*u.kpc/u.Myr,
                        **galcen_frame).decompose(galactic).value
    w0 = np.concatenate((x0, v0))
    return w0

def plot_mcmc_sample_orbit(p, R):
    w0 = _mcmc_sample_to_w0(p, R)
    orbit = potential.integrate_orbit(w0, dt=-0.5, t1=0., t2=freeze['t_back'])
    orbit_c,orbit_v = orbit.to_frame(coord.Galactic, **galcen_frame)

    fig,axes = pl.subplots(2,2,figsize=(12,8), sharex=True)

    # l,d
    axes[1,0].errorbar(obs_c.l.wrap_at(180*u.deg).degree, data['distance'].value, err['distance'].value, 
                       ls='none', marker='.', ecolor='#aaaaaa')
    l, = axes[1,0].plot(orbit_c.l.wrap_at(180*u.deg).degree, orbit_c.distance.value, marker=None)
    col = l.get_color()
    
    # l,b
    axes[0,0].scatter(obs_c.l.wrap_at(180*u.deg).degree, obs_c.b.degree, marker='.', c='k')
    axes[0,0].plot(orbit_c.l.wrap_at(180*u.deg).degree, orbit_c.b.degree, marker=None, c=col)

    # l,mul
    axes[0,1].errorbar(obs_c.l.wrap_at(180*u.deg).degree, data['mul'].value, err['mul'].value, 
                       ls='none', marker='.', ecolor='#aaaaaa')
    axes[0,1].plot(orbit_c.l.wrap_at(180*u.deg).degree, orbit_v[0].value, marker=None, c=col)

    # l,mub
    axes[1,1].errorbar(obs_c.l.wrap_at(180*u.deg).degree, data['mub'].value, err['mub'].value, 
                       ls='none', marker='.', ecolor='#aaaaaa')
    axes[1,1].plot(orbit_c.l.wrap_at(180*u.deg).degree, orbit_v[1].value, marker=None, c=col)
    
    axes[1,0].set_xlabel('$l$ [deg]')
    axes[1,1].set_xlabel('$l$ [deg]')
    
    fig.tight_layout()
    return fig,axes

In [ ]:
freeze = dict()
# these estimated from the plots
freeze['phi2_sigma'] = np.radians(0.9)
freeze['d_sigma'] = 0.15
freeze['vr_sigma'] = (1.5*u.km/u.s).decompose(galactic).value 
freeze['t_forw'] = 0.
freeze['t_back'] = -68.
for k,v in potential.parameters.items():
    freeze['potential_{}'.format(k)] = v.value

idx = data['phi1'].argmin()
p0_guess = [data['phi2'].radian[idx],
            data['distance'].decompose(galactic).value[idx],
            data['mul'].decompose(galactic).value[idx],
            data['mub'].decompose(galactic).value[idx],
            data['vr'].decompose(galactic).value[idx]]

In [ ]:
fig,axes = plot_mcmc_sample_orbit(p0_guess, R)

derp = orbitfit.rotate_sph_coordinate(coord.SphericalRepresentation(lon=data['phi1'][idx][None], 
                                                                    lat=data['phi2'][idx][None],
                                                                    distance=data['distance'][idx][None]), R.T)
g = coord.Galactic(derp)
axes[0,0].plot(g.l.degree, g.b.degree, marker='o', color='r', alpha=0.4)

In [ ]:
pl.plot(obs_c.l.degree, obs_v[2].to(u.km/u.s), ls='none')